# Human Activity Recognition - SSL JOINT Consistency

Last Update : 31 July 2019

In [1]:
N_THREADS = 8
# Nota Bene : notebooks don't deallocate GPU memory
IS_FORCE_CPU = False # can also be set in the trainer

## Environment

In [2]:
cd ..

/conv


In [3]:
%autosave 600
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# CENTER PLOTS
from IPython.core.display import HTML
display(HTML(""" <style> .output_png {display: table-cell; text-align: center; margin:auto; }
.prompt display:none;}  </style>"""))

import os
if IS_FORCE_CPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = ""

import sys
sys.path.append("notebooks")

import numpy as np
import matplotlib.pyplot as plt
from functools import partial
import pandas as pd
import h5py


import torch
torch.set_num_threads(N_THREADS)

Autosaving every 600 seconds


# Dataset

In [4]:
from skssl.transformers.neuralproc.datasplit import CntxtTrgtGetter, GetRandomIndcs, get_all_indcs
from utils.data.tsdata import get_timeseries_dataset, SparseMultiTimeSeriesDataset

get_cntxt_trgt_test = CntxtTrgtGetter(contexts_getter=GetRandomIndcs(min_n_indcs=0.1, max_n_indcs=0.5),
                                     targets_getter=get_all_indcs,
                                     is_add_cntxts_to_trgts=False)  # don't context points to tagrtes

get_cntxt_trgt_feat = CntxtTrgtGetter(contexts_getter=get_all_indcs,
                                     targets_getter=get_all_indcs,
                                     is_add_cntxts_to_trgts=False)  # don't context points to tagrtes

get_cntxt_trgt = CntxtTrgtGetter(contexts_getter=GetRandomIndcs(min_n_indcs=0.01, max_n_indcs=0.99),
                                 targets_getter=GetRandomIndcs(min_n_indcs=0.5, max_n_indcs=0.99),
                                 is_add_cntxts_to_trgts=False)  # don't context points to tagrtes

In [5]:
data_both = get_timeseries_dataset("har")(split="both")

def cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=False):
    def mycollate(batch):
        min_length = min([v.size(0) for b in batch for k,v in b[0].items() if "X" in k])
        # chose first min_legth of each (assumes that randomized)
        
        batch = [({k:v[:min_length, ...] for k,v in b[0].items()}, b[1]) for b in batch]        
        collated = torch.utils.data.dataloader.default_collate(batch)
        
        X = collated[0]["X"]
        y = collated[0]["y"]
        
        if is_repeat_batch:
            
            X = torch.cat([X,X], dim=0)
            y = torch.cat([y,y], dim=0)
            collated[1] = torch.cat([collated[1], collated[1]], dim=0) # targets
        
        collated[0]["X"], collated[0]["y"], collated[0]["X_trgt"], collated[0]["y_trgt"] = get_cntxt_trgt(X, y)
        
        return collated
    return mycollate

In [6]:
X_DIM = 1  # 1D spatial input (although actually 2 but the first is for sparse channels)
Y_DIM = data_both.data.shape[-1] # multiple channels
N_TARGETS = len(np.unique(data_both.targets))

sampling_percentages = [0.05, 0.1, 0.3, 0.5, 0.7, 1]
label_percentages = [N_TARGETS, N_TARGETS*2, 0.01, 0.05, 0.1, 0.3, 0.5, 1]

## Model

In [7]:
import torch.nn as nn
from skssl.transformers import GlobalNeuralProcess, NeuralProcessLoss, AttentiveNeuralProcess, NeuralProcessSSLLoss
from skssl.utils.helpers import rescale_range
from skssl.predefined import UnetCNN, CNN, MLP, SparseSetConv, SetConv, MlpRBF, GaussianRBF, BatchSparseSetConv
from skssl.transformers.neuralproc.datasplit import precomputed_cntxt_trgt_split
from utils.helpers import count_parameters

In [8]:
from copy import deepcopy

models = {}

min_std=5e-3

unet = partial(UnetCNN,
               Conv=torch.nn.Conv1d,
               Pool=torch.nn.MaxPool1d,
               upsample_mode="linear",
               n_layers=18,
               is_double_conv=True,
               is_depth_separable=True,
               Normalization=torch.nn.BatchNorm1d,
               is_chan_last=True,
               bottleneck=None,
               kernel_size=7,
               max_nchannels=256,
              is_force_same_bottleneck=True,
               _is_summary=True,
              )

kwargs = dict(x_dim=X_DIM, 
              y_dim=Y_DIM,
              min_std=min_std,
                n_tmp_queries=128,
                r_dim=64,
              keys_to_tmp_attn=partial(SetConv, RadialBasisFunc=GaussianRBF),
              TmpSelfAttn=unet,
              tmp_to_queries_attn=partial(SetConv, RadialBasisFunc=GaussianRBF),
              is_skip_tmp=False,
              is_use_x=False,
              get_cntxt_trgt=precomputed_cntxt_trgt_split,
              is_encode_xy=False,
             Classifier=partial(MLP, input_size=256+Y_DIM*4, output_size=N_TARGETS, 
                                dropout=0., hidden_size=128, n_hidden_layers=3, is_res=True))

models["ssl_classifier_gnp_large_shared_bottleneck"] = partial(GlobalNeuralProcess, **kwargs)

kwargs_bis = deepcopy(kwargs)
kwargs_bis["Classifier"] = None

models["transformer_gnp_large_shared_bottleneck"] = partial(GlobalNeuralProcess, **kwargs_bis)

In [9]:
from utils.helpers import count_parameters
for k,v in models.items():
    print(k, "- N Param:", count_parameters(v()))

ssl_classifier_gnp_large_shared_bottleneck - N Param: 1078238
transformer_gnp_large_shared_bottleneck - N Param: 1006936


In [10]:
def load_pretrained_(models, sampling_percentages):
    # ALREADY INITALIZE TO BE ABLE TO LOAD
    models["ssl_classifier_gnp_large_shared_bottleneck"] = partial(GlobalNeuralProcess, **kwargs)()

    kwargs_bis = deepcopy(kwargs)
    kwargs_bis["Classifier"] = None

    models["transformer_gnp_large_shared_bottleneck"] = partial(GlobalNeuralProcess, **kwargs_bis)()

    # load all transformers
    loaded_models = {}
    for sampling_perc in sampling_percentages:
        for k, m in models.items():
            if "transformer" not in k:
                continue

            out = train_models_({"{}%har".format(int(sampling_perc*100)): 
                                                (None, None)}, 
                                  {k :m },
                                   chckpnt_dirname=chckpnt_dirname_old,
                                seed=None,
                                   is_retrain=False)

            pretrained_model = out[list(out.keys())[0]].module_
            model_dict = models[k.replace("transformer", "ssl_classifier")].state_dict()
            model_dict.update(pretrained_model.state_dict())
            models[k.replace("transformer", "ssl_classifier")].load_state_dict(model_dict)

# Hyperparameter Optimisation


In [11]:
from ntbks_helpers import train_models_
from skorch.dataset import CVSplit
from utils.data.ssldata import get_train_dev_test_ssl
import random

N_EPOCHS = 100 
BATCH_SIZE = 32
IS_RETRAIN = False # if false load precomputed
chckpnt_dirname_old="results/challenge/har/"
chckpnt_dirname="results/challenge/har_new/"

from skssl.utils.helpers import HyperparameterInterpolator

n_steps_per_epoch = len(data_both)//BATCH_SIZE
get_lambda_clf=HyperparameterInterpolator(1, 10, N_EPOCHS*n_steps_per_epoch, mode="linear")

# No Augment

In [12]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.1]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              dev_size=0,
                                                              seed=random.randint(0,10000),
                                                              is_augment=False)
            print(len(data_train))

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run): 
                                                (data_train, data_test)}, 
                                  {k + "_finetune_noaug" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: 0.5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622
7352

--- Loading har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_noaug ---

har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_noaug best epoch: 20 val_loss: 0.27427897649091754

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622
7352

--- Loading har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_noaug ---

har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_noaug best epoch: 8 val_loss: 0.18001823478688497

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622
7352

--- Loading har100%_lab10%_run2/ssl_classifier_gnp_large_shared_bottleneck_f

In [13]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.934736   

                                                                              \
                                                                         std   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.001674   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.932813   

                                                                             \
                                                                        25%   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.93417   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.935528   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.935697   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.935867

# All

In [14]:
from skorch.callbacks import Freezer, LRScheduler


data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.1]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              dev_size=0,
                                                              seed=random.randint(0,10000),
                                                              is_augment=True)
            print(len(data_train))

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run): 
                                                (data_train, data_test)}, 
                                  {k + "_finetune" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: 0.5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622
12504

--- Loading har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune ---

har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune best epoch: 1 val_loss: 0.1426355674544359

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622
12504

--- Loading har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune ---

har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune best epoch: 1 val_loss: 0.14788568104815167

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622
12504

--- Loading har100%_lab10%_run2/ssl_classifier_gnp_large_shared_bottleneck_finetune ---

har100%_la

In [15]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.951929   

                                                                              \
                                                                         std   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.004755   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.947065   

                                                                             \
                                                                        25%   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.94961   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.952155   

                                                                             \
                                                                        75%   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.95436   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.956566

## Without Neg Consistency

In [16]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.1]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              dev_size=0,
                                                              seed=random.randint(0,10000),
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run):  
                                                (data_train, data_test)}, 
                                  {k + "_finetune_nonegcons" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 1,  # both do something similar
                                                     get_lambda_neg_cons=lambda: 0.,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_nonegcons ---

har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_nonegcons best epoch: 1 val_loss: 0.16748589485590443

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_nonegcons ---

har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_nonegcons best epoch: 2 val_loss: 0.1616656260042232

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab10%_run2/ssl_classifier_gnp_large_shared_bottleneck_fi

In [17]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.949327   

                                                                             \
                                                                        std   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.00193   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.947743   

                                                                              \
                                                                         25%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.948252   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.948761   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.950119   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.951476

# No Entropy

In [18]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.1]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              dev_size=0,
                                                              seed=random.randint(0,10000),
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run):
                                                (data_train, data_test)}, 
                                  {k + "_finetune_noent" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 0,  # both do something similar
                                                     get_lambda_neg_cons=lambda: 1.,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_noent ---

har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_noent best epoch: 1 val_loss: 0.15668519885409757

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_noent ---

har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_noent best epoch: 1 val_loss: 0.1925992108135756

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab10%_run2/ssl_classifier_gnp_large_shared_bottleneck_finetune_noent ---

In [19]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.945368   

                                                                              \
                                                                         std   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.001796   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.943332   

                                                                             \
                                                                        25%   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.94469   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.946047   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.946386   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.946725

## No Unsup

In [20]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.1]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              seed=random.randint(0,10000),
                                                              dev_size=0,
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run):
                                                (data_train, data_test)}, 
                                  {k + "_finetune_nounsup" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 0,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: .5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_nounsup ---

har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_nounsup best epoch: 1 val_loss: 0.15670357431684714

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_nounsup ---

har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_nounsup best epoch: 1 val_loss: 0.16049837192358557

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab10%_run2/ssl_classifier_gnp_large_shared_bottleneck_finetune_

In [21]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.952607   

                                                                              \
                                                                         std   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.003738   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.948761   

                                                                              \
                                                                         25%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.950797   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.952833   

                                                                             \
                                                                        75%   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.95453   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.956227

#  SSL Only

In [12]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.1]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc,
                                                              seed=random.randint(0,10000),
                                                              dev_size=0,
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run):
                                                (data_train, data_test)}, 
                                  {k + "_finetune_sslonly" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=True,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: .5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                               seed=None,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Training har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_sslonly ---



  epoch    train_loss    valid_acc    valid_loss    cp      dur
-------  ------------  -----------  ------------  ----  -------
      1        0.4417       0.9179        0.2222     +  81.2856


      2        0.2254       0.9365        0.2180     +  81.4742


      3        0.2322       0.9175        0.3426        79.5648


      4        0.1684       0.9233        0.4265        80.1608


...

     18        0.0342       0.9427        0.2807        55.6552


     19        0.1476       0.9539        0.2480        55.3252


     20        0.0428       0.9365        0.4001        54.9211


     21        0.4555       0.9498        0.2313        55.4418


Stopping since valid_acc has not improved in the last 15 epochs.
Re-initializing optimizer.
har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_sslonly best epoch: 7 val_loss: 0.15977995857611565


In [13]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%             2.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.960299   

                                                                             \
                                                                        std   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.00096   

                                                                             \
                                                                        min   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.95962   

                                                                              \
                                                                         25%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.959959   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.960299   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.960638   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.960977

## SUp Only

In [14]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

random.seed(1)

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.1]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")
            
            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc,
                                                              seed=random.randint(0,10000),
                                                              dev_size=0,
                                                              is_augment=True)


            # add test as unlabeled data
            data_train.data = data_train.data[(data_train.targets!=-1).squeeze()]
            data_train.targets = data_train.targets[(data_train.targets!=-1).squeeze()]

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run):
                                                (data_train, data_test)}, 
                                  {k + "_finetune_suponly" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=True,
                                                    get_lambda_unsup=lambda: 0,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: .5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                    seed=None,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Training har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_suponly ---



  epoch    train_loss    valid_acc    valid_loss    cp      dur
-------  ------------  -----------  ------------  ----  -------
      1        1.7205       0.9420        0.1594     +  17.2882


      2        0.5487       0.9522        0.1119     +  16.9470


      3        0.5105       0.9352        0.2376        17.3083


      4        0.5581       0.9491        0.1546        17.1064


...

     17        0.1470       0.9539        0.2167        17.5550


     18        0.0942       0.9576        0.2046        17.5022


     19        0.0968       0.9600        0.2575        17.1090


     20        0.1680       0.9494        0.2645        16.5031


Stopping since valid_acc has not improved in the last 15 epochs.
Re-initializing optimizer.
har100%_lab10%_run2/ssl_classifier_gnp_large_shared_bottleneck_finetune_suponly best epoch: 6 val_loss: 0.1170268439825891


In [15]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.963918   

                                                                              \
                                                                         std   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.004945   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.958263   

                                                                              \
                                                                         25%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.962165   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.966067   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.966746   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.967424

# Sup Only No Scaling

In [14]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.1]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc,
                                                              seed=random.randint(0,10000),
                                                              dev_size=0,
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = data_train.data[(data_train.targets!=-1).squeeze()]
            data_train.targets = data_train.targets[(data_train.targets!=-1).squeeze()]

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run):
                                                (data_train, data_test)}, 
                                  {k + "_finetune_sup_vanilla" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda : 1,
                                                    n_max_elements=None,
                                                    label_perc=None, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=True,
                                                    get_lambda_unsup=lambda: 0,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=True,
                                               seed=None,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))


--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Training har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_sup_vanilla ---



  epoch    train_loss    valid_acc    valid_loss    cp      dur
-------  ------------  -----------  ------------  ----  -------
      1        0.2326       0.9233        0.2089     +  17.7597


      2        0.0824       0.9535        0.1454     +  17.5394


      3        0.0482       0.9610        0.1342     +  17.2043


      4        0.0490       0.9481        0.1779        17.8247


...

     14        0.0239       0.9403        0.2933        25.1758


     15        0.0174       0.9369        0.3352        24.5751


     16        0.0406       0.9471        0.2295        24.6295


     17        0.0167       0.9528        0.2350        24.9286


Stopping since valid_acc has not improved in the last 15 epochs.
Re-initializing optimizer.
har100%_lab10%_run2/ssl_classifier_gnp_large_shared_bottleneck_finetune_sup_vanilla best epoch: 3 val_loss: 0.12915067612861947


In [15]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.962787   

                                                                              \
                                                                         std   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.005194   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.956905   

                                                                              \
                                                                         25%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.960808   

                                                                             \
                                                                        50%   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.96471   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.965728   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.966746

# No Lambda CLF

In [24]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.1]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              dev_size=0,
                                                              seed=random.randint(0,10000),
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run): 
                                                (data_train, data_test)}, 
                                  {k + "_finetune_nolambda" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: 1,
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: .5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_nolambda ---

har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_nolambda best epoch: 1 val_loss: 0.16846257951812668

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_nolambda ---

har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_nolambda best epoch: 1 val_loss: 0.14787215202754517

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab10%_run2/ssl_classifier_gnp_large_shared_bottleneck_finet

In [25]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.956566   

                                                                              \
                                                                         std   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.008333   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.947404   

                                                                              \
                                                                         25%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.953003   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.958602   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.961147   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.963692

## Without Label Scaling

In [26]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.1]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              dev_size=0,
                                                              seed=random.randint(0,10000),
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run): 
                                                (data_train, data_test)}, 
                                  {k + "_finetune_nolabscale" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=None, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: 0.5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_nolabscale ---

har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_nolabscale best epoch: 18 val_loss: 0.2315054115376984

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_nolabscale ---

har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_nolabscale best epoch: 2 val_loss: 0.18339065070230307

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab10%_run2/ssl_classifier_gnp_large_shared_bottlene

In [27]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.953399   

                                                                              \
                                                                         std   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.007567   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.947743   

                                                                              \
                                                                         25%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.949101   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.950458   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.956227   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.961995

## No Element Scaling

In [28]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.1]:

            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              dev_size=0,
                                                              seed=random.randint(0,10000),
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run): 
                                                (data_train, data_test)}, 
                                  {k + "_finetune_noelemscale" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=None,
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: 0.5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_noelemscale ---

har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_noelemscale best epoch: 7 val_loss: 0.1779502336963541

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_noelemscale ---

har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_noelemscale best epoch: 7 val_loss: 0.20060293440661897

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab10%_run2/ssl_classifier_gnp_large_shared_bottl

In [29]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.951815   

                                                                              \
                                                                         std   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.001889   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.949779   

                                                                              \
                                                                         25%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.950967   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.952155   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.952833   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 10% 100%         0.953512

In [29]:
for k,t in data_trainers.items(): 
    for e, h in enumerate(t.history[::-1]):
        if h["valid_acc_best"]:
            print(k, "epoch:", len(t.history)-e, 
                  "val_loss:", h["valid_loss"], 
                  "val_acc:", h["valid_acc"])
            break

har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_noelemscale epoch: 7 val_loss: 0.1779502336963541 val_acc: 0.9429928741092637
har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_noelemscale epoch: 7 val_loss: 0.20060293440661897 val_acc: 0.9399389209365456
har100%_lab10%_run2/ssl_classifier_gnp_large_shared_bottleneck_finetune_noelemscale epoch: 7 val_loss: 0.15404326194903015 val_acc: 0.9535120461486257


## No Finetuning

In [30]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}
models["ssl_classifier_gnp_large_shared_bottleneck"] = partial(GlobalNeuralProcess, **kwargs)

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.1]:

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              dev_size=0,
                                                              seed=random.randint(0,10000),
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run): 
                                                (data_train, data_test)}, 
                                  {k + "" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: 0.5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck ---

har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck best epoch: 2 val_loss: 0.2708716995238013

--- Loading har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck ---

har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck best epoch: 5 val_loss: 0.1961393239824964

--- Loading har100%_lab10%_run2/ssl_classifier_gnp_large_shared_bottleneck ---

har100%_lab10%_run2/ssl_classifier_gnp_large_shared_bottleneck best epoch: 2 val_loss: 0.19988441305480895


In [31]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy            \
                                                              count      mean   
models                                     lab data sample                      
ssl_classifier_gnp_large_shared_bottleneck 10% 100%             3.0  0.931795   

                                                                               \
                                                                std       min   
models                                     lab data sample                      
ssl_classifier_gnp_large_shared_bottleneck 10% 100%         0.00622  0.925008   

                                                                               \
                                                                25%       50%   
models                                     lab data sample                      
ssl_classifier_gnp_large_shared_bottleneck 10% 100%         0.92908  0.933152   

                                                                                
                                                                 75%       max  
models                                     lab data sample                      
ssl_classifier_gnp_large_shared_bottleneck 10% 100%         0.935188  0.937224

In [31]:
#0.9644 with both supervised and unsupervised with finetuning

for k,t in data_trainers.items(): 
    for e, h in enumerate(t.history[::-1]):
        if h["valid_acc_best"]:
            print(k, "epoch:", len(t.history)-e, 
                  "val_loss:", h["valid_loss"], 
                  "val_acc:", h["valid_acc"])
            break

har100%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck epoch: 2 val_loss: 0.2708716995238013 val_acc: 0.9121140142517815
har100%_lab10%_run1/ssl_classifier_gnp_large_shared_bottleneck epoch: 5 val_loss: 0.1961393239824964 val_acc: 0.9307770614183916
har100%_lab10%_run2/ssl_classifier_gnp_large_shared_bottleneck epoch: 2 val_loss: 0.19988441305480895 val_acc: 0.9270444519850696


# Evaluation Sampling Percentage

In [32]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in sampling_percentages:
        for label_perc in [1]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              dev_size=0,
                                                              seed=random.randint(0,10000),
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run): 
                                                (data_train, data_test)}, 
                                  {k + "_finetune" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: 0.5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 5%har/transformer_gnp_large_shared_bottleneck ---

5%har/transformer_gnp_large_shared_bottleneck best epoch: 98 val_loss: -2.164531707763672

--- Loading har5%_lab100%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune ---

har5%_lab100%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune best epoch: 5 val_loss: 0.6508865955121079

--- Loading 10%har/transformer_gnp_large_shared_bottleneck ---

10%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -2.98053588682008

--- Loading har10%_lab100%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune ---

har10%_lab100%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune best epoch: 11 val_loss: 0.37507563022431145

--- Loading 30%har/transformer_gnp_large_shared_bottleneck ---

30%har/transformer_gnp_large_shared_bottleneck best epoch: 87 val_loss: -4.5797279249117215

--- Loading har30%_lab100%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune ---

har30%_lab100%_run0/ssl_classifier

In [33]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                       count   
models                                             lab  data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 100% 10%              3.0   
                                                        100%             3.0   
                                                        30%              3.0   
                                                        5%               3.0   
                                                        50%              3.0   
                                                        70%              3.0   

                                                                               \
                                                                         mean   
models                                             lab  data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 100% 10%          0.902047   
                                                        100%         0.976021   
                                                        30%          0.957245   
                                                        5%           0.820269   
                                                        50%          0.963353   
                                                        70%          0.975908   

                                                                               \
                                                                          std   
models                                             lab  data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 100% 10%          0.000706   
                                                        100%         0.007296   
                                                        30%          0.000588   
                                                        5%           0.008146   
                                                        50%          0.007907   
                                                        70%          0.001796   

                                                                               \
                                                                          min   
models                                             lab  data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 100% 10%          0.901256   
                                                        100%         0.968782   
                                                        30%          0.956905   
                                                        5%           0.810994   
                                                        50%          0.954530   
                                                        70%          0.974550   

                                                                               \
                                                                          25%   
models                                             lab  data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 100% 10%          0.901765   
                                                        100%         0.972345   
                                                        30%          0.956905   
                                                        5%           0.817272   
                                                        50%          0.960129   
                                                        70%          0.974890   

                                                                               \
                                                                          50%   
models                                             lab  data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 100% 10%          0.902273   
                                                        100%         0.975908   
                      

In [33]:
for k,t in data_trainers.items(): 
    for e, h in enumerate(t.history[::-1]):
        if h["valid_acc_best"]:
            print(k, "epoch:", len(t.history)-e, 
                  "val_loss:", h["valid_loss"], 
                  "val_acc:", h["valid_acc"])
            break

har5%_lab100%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune epoch: 5 val_loss: 0.6508865955121079 val_acc: 0.7553444180522565
har10%_lab100%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune epoch: 11 val_loss: 0.37507563022431145 val_acc: 0.8961655921275874
har30%_lab100%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune epoch: 10 val_loss: 0.23837404603831922 val_acc: 0.9491007804546997
har50%_lab100%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune epoch: 1 val_loss: 0.1840641939482286 val_acc: 0.9474041398031897
har70%_lab100%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune epoch: 10 val_loss: 0.0867358241843336 val_acc: 0.9752290464879538
har100%_lab100%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune epoch: 32 val_loss: 0.09407840634266076 val_acc: 0.9799796403121819
har5%_lab100%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune epoch: 4 val_loss: 0.67058725331976 val_acc: 0.7451645741431965
har10%_lab100%_run1/ssl_classi

# Evaluation Label Percentage

In [34]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [0.5]:
        for label_perc in label_percentages:
            is_retrain = IS_RETRAIN
            if label_perc == 1: #already computed previous cell
                is_retrain = False

            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              dev_size=0,
                                                              seed=random.randint(0,10000),
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run): 
                                                (data_train, data_test)}, 
                                  {k + "_finetune" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: 0.5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))


--- Loading 50%har/transformer_gnp_large_shared_bottleneck ---

50%har/transformer_gnp_large_shared_bottleneck best epoch: 72 val_loss: -5.531349883959131

--- Loading har50%_lab600%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune ---

har50%_lab600%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune best epoch: 1 val_loss: 1.7390090251802226

--- Loading 50%har/transformer_gnp_large_shared_bottleneck ---

50%har/transformer_gnp_large_shared_bottleneck best epoch: 72 val_loss: -5.531349883959131

--- Loading har50%_lab1200%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune ---

har50%_lab1200%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune best epoch: 4 val_loss: 1.7610424022816948

--- Loading 50%har/transformer_gnp_large_shared_bottleneck ---

50%har/transformer_gnp_large_shared_bottleneck best epoch: 72 val_loss: -5.531349883959131

--- Loading har50%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune ---

har50%_lab1%_run0/ssl_classifier


--- Loading har50%_lab100%_run2/ssl_classifier_gnp_large_shared_bottleneck_finetune ---

har50%_lab100%_run2/ssl_classifier_gnp_large_shared_bottleneck_finetune best epoch: 2 val_loss: 0.10383275301932965


In [35]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                        count   
models                                             lab   data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%    50%              3.0   
                                                   10%   50%              3.0   
                                                   100%  50%              3.0   
                                                   1200% 50%              3.0   
                                                   30%   50%              3.0   
                                                   5%    50%              3.0   
                                                   50%   50%              3.0   
                                                   600%  50%              3.0   

                                                                                \
                                                                          mean   
models                                             lab   data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%    50%          0.868793   
                                                   10%   50%          0.944237   
                                                   100%  50%          0.963353   
                                                   1200% 50%          0.337066   
                                                   30%   50%          0.959394   
                                                   5%    50%          0.937790   
                                                   50%   50%          0.962787   
                                                   600%  50%          0.321570   

                                                                                \
                                                                           std   
models                                             lab   data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%    50%          0.001530   
                                                   10%   50%          0.001530   
                                                   100%  50%          0.007907   
                                                   1200% 50%          0.020202   
                                                   30%   50%          0.001371   
                                                   5%    50%          0.007514   
                                                   50%   50%          0.003399   
                                                   600%  50%          0.016940   

                                                                                \
                                                                           min   
models                                             lab   data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%    50%          0.867323   
                                                   10%   50%          0.942654   
                                                   100%  50%          0.954530   
                                                   1200% 50%          0.314218   
                                                   30%   50%          0.958602   
                                                   5%    50%          0.932474   
                                                   50%   50%          0.959281   
                                                   600%  50%          0.307431   

                                                                                \
                                                                           25%   
models                                             lab   data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%    50%          0.868001   
                                                   10%   50%          0.943502   
                                                   100%  5

In [37]:
# if bad has to try freezing again and smaller params
for k,t in data_trainers.items(): 
    for e, h in enumerate(t.history[::-1]):
        if h["valid_acc_best"]:
            print(k, "epoch:", len(t.history)-e, 
                  "val_loss:", h["valid_loss"], 
                  "val_acc:", h["valid_acc"])
            break

har50%_lab600%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune epoch: 3 val_loss: 1.783490130688564 val_acc: 0.3169324737020699
har50%_lab1200%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune epoch: 21 val_loss: 1.7878198685142845 val_acc: 0.3525619273837801
har50%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune epoch: 11 val_loss: 1.0262672793553742 val_acc: 0.8673227010519172
har50%_lab5%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune epoch: 9 val_loss: 0.25123960873940704 val_acc: 0.9324737020699015
har50%_lab10%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune epoch: 15 val_loss: 0.3563840006180122 val_acc: 0.9443501866304717
har50%_lab30%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune epoch: 32 val_loss: 0.2326741750093893 val_acc: 0.9586019681031558
har50%_lab50%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune epoch: 17 val_loss: 0.1826831086999811 val_acc: 0.9630132337970818
har50%_lab100%_run0/ssl_classifier_

* 0.9304 best without n max elements
* 0.9277: jsd | no 0.1 scale | n_max_elements | 100 sampels | 0.05 entropies
* 0.9857 : jsd | 0.1 scale | n_max_elements | 100 sampels | 0.05 entropies | no freeze
* 0.9623 : jsd | no scale | n_max_elements | 100 sampels | 0.01 entropies | no freeze | no pretrain | [0.01,0.5] | linear interpolator (1,5)


* 0.9671 : jsd | no scale ? | n_max_elements | 100 sampels | 0.05 entropies | no freeze | no pretrain | [0.1,0.5]
* 0.9365 : jsd | no scale | n_max_elements | 100 sampels | 0.05 entropies | no freeze | no pretrain | [0.01,0.5] | linear interpolator
* 0.9824 : jsd | no scale | n_max_elements | 100 sampels | 0.01 entropies | no freeze | no pretrain | [0.01,0.5] | linear interpolator


* 0.9844 : jsd | no 0.1 scale | n_max_elements | 100 sampels | 0.05 entropies | no freeze
* 0.9817 : jsd | no 0.1 scale | n_max_elements | 100 sampels | 0.05 entropies | no freeze | cntxt [0.01,0.5]


* 0.9627 : jsd | no scale | n_max_elements | 100 sampels | 0.01 entropies | no freeze | no pretrain | [0.01,0.5]
* 0.9572 : jsd | no scale | n_max_elements | 100 sampels | 0.01 entropies | no freeze | no pretrain | [0.01,0.9] | linear interpolator


* 0.9321: jsd | no 0.1 scale | n_max_elements | 100 sampels
* 0.9365: jsd | no 0.1 scale | n_max_elements | 100 sampels | 0.1 entropies


* 0.9450 : jsd | 0.2 scale | n_max_elements | 100 sampels | 0.05 entropies | no freeze | no pretrain | [0.01,0.5]
* 0.9315 : jsd | no scale | n_max_elements | 100 sampels | 0.05 entropies | no freeze | no pretrain | [0.01,0.5]

In [28]:
# load all transformers
loaded_models = {}
for sampling_perc in sampling_percentages:
    for k, m in models.items():
        if "transformer" not in k:
            continue
            
        out = train_models_({"{}%har".format(int(sampling_perc*100)): 
                                            (None, None)}, 
                              {k :m },
                               chckpnt_dirname=chckpnt_dirname,
                               is_retrain=False)
        
        pretrained_model = out[list(out.keys())[0]].module_
        model_dict = models[k.replace("transformer", "ssl_classifier")].state_dict()
        model_dict.update(pretrained_model.state_dict())
        models[k.replace("transformer", "ssl_classifier")].load_state_dict(model_dict)
        


--- Loading 5%har/transformer_gnp_large_shared_bottleneck ---

5%har/transformer_gnp_large_shared_bottleneck best epoch: 98 val_loss: -2.164531707763672

--- Loading 10%har/transformer_gnp_large_shared_bottleneck ---

10%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -2.98053588682008

--- Loading 30%har/transformer_gnp_large_shared_bottleneck ---

30%har/transformer_gnp_large_shared_bottleneck best epoch: 87 val_loss: -4.5797279249117215

--- Loading 50%har/transformer_gnp_large_shared_bottleneck ---

50%har/transformer_gnp_large_shared_bottleneck best epoch: 72 val_loss: -5.531349883959131

--- Loading 70%har/transformer_gnp_large_shared_bottleneck ---

70%har/transformer_gnp_large_shared_bottleneck best epoch: 98 val_loss: -7.103462460897501

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622


In [29]:
from torch.distributions import Normal, Categorical, kl_divergence

In [30]:
t1 = torch.tensor([0.2, 0.8], requires_grad=True)
t2 = torch.tensor([0.7, 0.3], requires_grad=True)
#torch.softmax(t2, -1)
#torch.softmax(t1, -1)

In [31]:
M = (t1 + t2) / 2

In [ ]:
def jensen_shannon_div(p1, p2):
    p_avg = (p1 + p2) / 2
    mask = (p_avg != 0).float()
    # set to 0 p when M is 0 (because mean can only be 0 is vectors weree, but
    # this is not the case due to numerical issues)
    M = Categorical(probs=p_avg)
    return ((kl_divergence(Categorical(probs=p1 * mask), M) +
             kl_divergence(Categorical(probs=p2 * mask), M)) / 2)

def yann_div(t1, t2):
    M = (t1 + t2) / 2
    return torch.min(kl_divergence(Categorical(probs=t1), Categorical(M)) + 
               kl_divergence(Categorical(probs=t2), Categorical(M)))

def csiszar_dist(t1, t2):
    M = (t1 + t2) / 2
    return ((kl_divergence(Categorical(M), Categorical(probs=t1)
                ) + kl_divergence(Categorical(M), Categorical(probs=t2)))/2)#**0.5

def total_var(t1, t2):
    return (t1 - t2).abs().sum(-1) / 2

def bhattacharyya_dist(t1, t2):
    return -torch.log((t1 * t2).sqrt().sum(-1))

def hellinger_dist(t1, t2):
    return (t1.sqrt() - t2.sqrt()).pow(2).sum(-1).sqrt() / (2**0.5)

In [ ]:
import math
math.log(2)

In [ ]:
for t1,t2 in [([0., 1], [1, 0.]), 
              ([0.5, 0.5], [0.4, 0.6]), 
              ([0.5, 0.5], [0.5, 0.5]), 
              ([0.4, 0.6], [0.3, 0.7]), 
              ([1-1e-50, 1e-50], [1e-50, 1-1e-50]), 
              ([0.1, 0.1, 0.8], [0.2, 0.2, 0.6]), 
              ([0.1, 0.1, 0.8], [0.6, 0.2, 0.2])]:
    print()
    print(t1, t2)
    print("yd", yann_div(torch.tensor(t1), torch.tensor(t2)).item())
    print("cd", csiszar_dist(torch.tensor(t1), torch.tensor(t2)).item())
    print("tv", total_var(torch.tensor(t1), torch.tensor(t2)).item())
    print("jsd", jensen_shannon_div(torch.tensor(t1), torch.tensor(t2)).item())
    print("bd", bhattacharyya_dist(torch.tensor(t1), torch.tensor(t2)).item())
    print("hd", hellinger_dist(torch.tensor(t1), torch.tensor(t2)).item())

In [ ]:
t1.pow(2)

In [ ]:
k.backward()

t1.grad

In [ ]:
t2.grad